# Understanding Ontologies with Cognee

This notebook demonstrates how to work with ontologies in scientific research using the Cognee framework. We'll explore how ontologies can enhance our understanding and querying of scientific papers.

## What is an Ontology?

An ontology is a formal representation of knowledge that defines:
- Concepts within a domain
- Relationships between concepts
- Properties and attributes
- Rules and constraints

Key terms:
- **Classes**: Categories or types (e.g., Disease, Symptom)
- **Instances**: Specific examples of classes (e.g., Type 2 Diabetes)
- **Properties**: Relationships between classes/instances (e.g., hasSymptom)
- **Axioms**: Logical statements defining relationships

## Setup

First, let's install the required packages and set up our environment:

In [1]:
# Install required package
# !pip install cognee

In [2]:
import os

# Set up OpenAI API key (required for Cognee's LLM functionality)
if "LLM_API_KEY" not in os.environ:
    os.environ["LLM_API_KEY"] = "your-api-key-here"  # Replace with your API key

In [3]:
# Import required libraries
import cognee
print(cognee.__version__)
from cognee.shared.logging_utils import get_logger

cognee.config.set_llm_model("gpt-4o-mini")
cognee.config.set_llm_provider("openai")
from cognee.api.v1.search import SearchType

logger = get_logger()


2025-10-22T18:01:55.986758 [info     ] Deleted old log file: /Users/daulet/Desktop/dev/cognee-claude/logs/2025-10-22_18-22-34.log [cognee.shared.logging_utils]

2025-10-22T18:01:56.875637 [info     ] Logging initialized            [cognee.shared.logging_utils] cognee_version=0.3.6-local database_path=/Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases graph_database_name= os_info='Darwin 24.5.0 (Darwin Kernel Version 24.5.0: Tue Apr 22 19:54:43 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T8132)' python_version=3.10.11 relational_config=cognee_db structlog_version=25.4.0 vector_config=lancedb

2025-10-22T18:01:56.876198 [info     ] Database storage: /Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases [cognee.shared.logging_utils]


0.3.6-local


## Creating the Pipeline

Let's create a pipeline that will:
1. Clean existing data
2. Process scientific papers
3. Apply ontological knowledge

In [4]:
async def run_pipeline(config=None):
    # Clean existing data
    await cognee.prune.prune_data()
    await cognee.prune.prune_system(metadata=True)
    
    # Set up path to scientific papers
    scientific_papers_dir = os.path.join(
        os.path.dirname(os.path.dirname(os.path.abspath("."))), 
        "cognee",
        "examples",
        "data", 
        "scientific_papers/"
    )
    
    # Add papers to the system
    await cognee.add(scientific_papers_dir)
    
    # Cognify with optional ontology
    return await cognee.cognify(config=config)

async def query_pipeline(questions):
    answers = []
    for question in questions:
        search_results = await cognee.search(
            query_type=SearchType.GRAPH_COMPLETION,
            query_text=question,
        )
        answers.append(search_results)
    return answers

## Running the Demo

Let's test our system with some medical questions, comparing results with and without ontological knowledge:

In [ ]:
from cognee.modules.ontology.rdf_xml.RDFLibOntologyResolver import RDFLibOntologyResolver
from cognee.modules.ontology.ontology_config import Config
# Test questions
questions = [
    "What are common risk factors for Type 2 Diabetes?",
    "What preventive measures reduce the risk of Hypertension?",
    "What symptoms indicate possible Cardiovascular Disease?",
    "What diseases are associated with Obesity?"
]

# Path to medical ontology
ontology_path = "../examples/python/ontology_input_example/enriched_medical_ontology_with_classes.owl"  # Update with your ontology path

config: Config = {
        "ontology_config": {
            "ontology_resolver": RDFLibOntologyResolver(ontology_file=ontology_path)
        }
    }

# Run with ontology
print("\n--- Results WITH ontology ---\n")
await run_pipeline(config=config)
answers_with = await query_pipeline(questions)
for q, a in zip(questions, answers_with):
    print(f"Q: {q}\nA: {a}\n")


2025-10-22T18:01:58.278557 [info     ] Ontology loaded successfully from file: ../examples/python/ontology_input_example/enriched_medical_ontology_with_classes.owl [OntologyAdapter]

2025-10-22T18:01:58.280303 [info     ] Lookup built: 4 classes, 50 individuals [OntologyAdapter]

2025-10-22T18:01:58.376936 [info     ] Loaded JSON extension          [cognee.shared.logging_utils]



--- Results WITH ontology ---




2025-10-22T18:01:58.421862 [info     ] Deleted Kuzu database files at /Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases/cognee_graph_kuzu [cognee.shared.logging_utils]

2025-10-22T18:02:01.325380 [info     ] Database deleted successfully. [cognee.shared.logging_utils]

Storage manager absolute path: /Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_cache

Deleting cache...             

✓ Cache deleted successfully! 


User bbafc85c-4a78-404e-a5ba-fd0283528287 has registered.



2025-10-22T18:02:02.808090 [info     ] Pipeline run started: `e4e8d210-231c-5518-9068-cf56e37634d0` [run_tasks_with_telemetry()]

2025-10-22T18:02:02.808489 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-10-22T18:02:02.808812 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-10-22T18:02:02.809810 [info     ] Pipeline run started: `e4e8d210-231c-5518-9068-cf56e37634d0` [run_tasks_with_telemetry()]

2025-10-22T18:02:02.810062 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-10-22T18:02:02.810401 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-10-22T18:02:02.821927 [info     ] Registered loader: pypdf_loader [cognee.infrastructure.loaders.LoaderEngine]

2025-10-22T18:02:02.822839 [info     ] Registered loader: text_loader [cognee.infrastructure.loaders.LoaderEngine]

2025-10-22T18:02:02.823183 [info     ] Registered loader: image_loader [cognee.infrastructure.loader

Q: What are common risk factors for Type 2 Diabetes?
A: ['Common risk factors for Type 2 Diabetes include:\n- Obesity\n- Hypertension (high blood pressure)\n- High cholesterol\n- Cardiovascular disease\n- Smoking\n- Family history of diabetes']

Q: What preventive measures reduce the risk of Hypertension?
A: ['Preventive measures to reduce the risk of hypertension include:\n1. Low sodium diet\n2. Moderate coffee consumption\n3. Regular exercise and physical activity.']

Q: What symptoms indicate possible Cardiovascular Disease?
A: ['Symptoms indicating possible Cardiovascular Disease include:\n- Chest pain\n- Shortness of breath\n- Fatigue']

Q: What diseases are associated with Obesity?
A: ['Diseases associated with obesity include:\n1. Cardiovascular disease\n2. Diabetes\n3. Hypertension\n4. High cholesterol\n5. High blood pressure\n6. Atrial fibrillation\n7. Heart disease\n8. Heart failure\n9. Coronary artery disease']




2025-10-22T18:03:27.048651 [info     ] Reconnecting to Kuzu database... [cognee.shared.logging_utils]

2025-10-22T18:03:27.093759 [info     ] Loaded JSON extension          [cognee.shared.logging_utils]


In [6]:
# Run without ontology
print("\n--- Results WITHOUT ontology ---\n")
await run_pipeline()
answers_without = await query_pipeline(questions)
for q, a in zip(questions, answers_without):
    print(f"Q: {q}\nA: {a}\n")


2025-10-22T18:03:02.112053 [info     ] Deleted Kuzu database files at /Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases/cognee_graph_kuzu [cognee.shared.logging_utils]



--- Results WITHOUT ontology ---




2025-10-22T18:03:04.157767 [info     ] Database deleted successfully. [cognee.shared.logging_utils]

Deleting cache...             

✓ Cache deleted successfully! 

2025-10-22T18:03:04.270250 [info     ] Pipeline run started: `be0945ca-b02d-57d3-8c55-2e66cb0e6465` [run_tasks_with_telemetry()]

2025-10-22T18:03:04.270711 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-10-22T18:03:04.271253 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-10-22T18:03:04.271827 [info     ] Pipeline run started: `be0945ca-b02d-57d3-8c55-2e66cb0e6465` [run_tasks_with_telemetry()]

2025-10-22T18:03:04.272160 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-10-22T18:03:04.272586 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-10-22T18:03:04.289999 [info     ] Reading PDF: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/TOJ-22-0073_152Mendoza.pdf [cognee.infrastructure.l

User 287cca05-43b5-48c2-90de-2e86072d27f0 has registered.



2025-10-22T18:03:04.576198 [info     ] Coroutine task completed: `ingest_data` [run_tasks_base]

2025-10-22T18:03:04.576640 [info     ] Coroutine task completed: `resolve_data_directories` [run_tasks_base]

2025-10-22T18:03:04.577015 [info     ] Pipeline run completed: `be0945ca-b02d-57d3-8c55-2e66cb0e6465` [run_tasks_with_telemetry()]

2025-10-22T18:03:04.580698 [info     ] Coroutine task completed: `ingest_data` [run_tasks_base]

2025-10-22T18:03:04.581235 [info     ] Coroutine task completed: `resolve_data_directories` [run_tasks_base]

2025-10-22T18:03:04.581502 [info     ] Pipeline run completed: `be0945ca-b02d-57d3-8c55-2e66cb0e6465` [run_tasks_with_telemetry()]

2025-10-22T18:03:04.586826 [info     ] Ontology file 'None' not found. No owl ontology will be attached to the graph. [OntologyAdapter]

2025-10-22T18:03:04.600355 [info     ] Pipeline run started: `8654ab86-b9d9-5cea-b22e-0732dd3d817d` [run_tasks_with_telemetry()]

2025-10-22T18:03:04.600702 [info     ] Coroutine task 

Q: What are common risk factors for Type 2 Diabetes?
A: ['Common risk factors for Type 2 Diabetes include smoking, obesity (especially with a high body mass index or waist circumference), physical inactivity, and poor dietary habits. Additionally, pre-existing chronic diseases such as high blood pressure and high blood cholesterol can increase the risk. Moderate coffee consumption may also affect the risk, as it has been linked with lower incidences of Type 2 Diabetes in some studies.']

Q: What preventive measures reduce the risk of Hypertension?
A: ['Preventive measures to reduce the risk of hypertension include moderate coffee consumption (typically defined as 1 to 4 cups per day), which is associated with a decreased risk of developing hypertension. Additionally, avoiding smoking can also mitigate risk as smoking increases the risk of cardiovascular diseases.']

Q: What symptoms indicate possible Cardiovascular Disease?
A: ['Symptoms that may indicate possible cardiovascular diseas

## Visualizing the Knowledge Graph

Let's visualize how our ontology connects different medical concepts:

In [7]:
import webbrowser
import os
from cognee.api.v1.visualize.visualize import visualize_graph
html = await visualize_graph()
home_dir = os.path.expanduser("~")
html_file = os.path.join(home_dir, "graph_visualization.html")
display(html_file)
webbrowser.open(f"file://{html_file}")


2025-10-22T18:03:55.296116 [info     ] Graph visualization saved as /Users/daulet/graph_visualization.html [cognee.shared.logging_utils]

2025-10-22T18:03:55.296647 [info     ] The HTML file has been stored on your home directory! Navigate there with cd ~ [cognee.shared.logging_utils]


'/Users/daulet/graph_visualization.html'

True

## Understanding the Results

The demonstration above shows how ontologies enhance our analysis by:

1. **Making Connections**: 
   - Linking related medical concepts even when not explicitly stated
   - Identifying relationships between symptoms, diseases, and risk factors

2. **Standardizing Terms**: 
   - Unifying different ways of referring to the same medical condition
   - Ensuring consistent terminology across documents

3. **Enabling Inference**: 
   - Drawing conclusions based on ontological relationships
   - Discovering implicit connections in the data

## Next Steps

To learn more about Cognee and ontologies:
1. Check out the [Cognee documentation](https://docs.cognee.ai/)
2. Explore more examples in the `examples` directory
3. Try creating your own domain-specific ontology

Remember to:
- Place your scientific papers in the appropriate directory
- Update the ontology path to point to your .owl file
- Replace the API key with your own OpenAI key

In [ ]:
# Only exit in interactive mode, not during GitHub Actions
import os

# Skip exit if we're running in GitHub Actions
if not os.environ.get('GITHUB_ACTIONS'):
    print("Exiting kernel to clean up resources...")
    os._exit(0)
else:
    print("Skipping kernel exit - running in GitHub Actions")

: 